In [ ]:
!pip install crewai
!pip install requests
!pip install langchain_google_genai
!pip install crewai-tools # Install the correct package



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 647.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of grpcio-tools to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.9/210.9 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.6/990.6 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.5/61.5 kB 5.8 MB/s eta 0:00:

In [ ]:
import os
import crewai
import warnings
warnings.filterwarnings('ignore')
from crewai import Agent, Task, Crew
# Importing crewAI tools
from crewai_tools import (
    SerperDevTool,
    WebsiteSearchTool,
    ScrapeWebsiteTool
)
from langchain_google_genai import ChatGoogleGenerativeAI


In [ ]:
from google.colab import userdata
# Retrieve your Google API key
google_api_key = userdata.get('Gemini_API')
os.environ["GOOGLE_API_KEY"] = google_api_key
# Retrieve your Serper API key
serper_api_key = userdata.get('SERPER_API_KEY')
os.environ["SERPER_API_KEY"] = serper_api_key

In [ ]:
## call the gemini models
gemini=ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                           verbose=True,
                           temperature=0.5,
                           google_api_key=google_api_key)


In [ ]:
# Instantiate tools
#docs_tool = DirectoryReadTool(directory='./blog-posts')
#file_tool = FileReadTool()
search_tool = SerperDevTool(api_key=serper_api_key)
#web_rag_tool = WebsiteSearchTool()
scrape_tool= ScrapeWebsiteTool()


In [ ]:
retriever_agent = Agent(
    role='Retriever',
    goal='Retrieve and summarize search results into a concise response.',
    backstory="""You are a retriever agent specialized in summarizing search results into a concise response
    based on RAG processing. Your role is to extract the most relevant information and provide a response
    within 10 lines.""",
    verbose=True,
    llm=gemini,
    tools=[search_tool, scrape_tool],
    allow_delegation=True
)

In [ ]:
rag_task = Task(
    description='Retrieve and summarize RAG-based output from search results',
    expected_output=(
        'A concise summary of search results in 10 lines or less, without including any website URLs.'
    ),
    agent=retriever_agent,
    tools=[search_tool, scrape_tool],
    llm=gemini
)

In [ ]:
dermatologist_agent = Agent(
        role='Dermatologist',
        goal='Provide accurate and helpful dermatology-related information and advice.',
        backstory="""You are a dermatologist specializing in providing information and advice on skin conditions,
        treatments, and skincare practices.""",
        verbose=True,
        llm=gemini,
        tools=[search_tool,scrape_tool],
        allow_delegation=True)

In [ ]:
# Define the Dermatology Query Task
dermatology_task = Task(
    description='Handle dermatology-related user queries with RAG and search',
    expected_output=(
        'A concise response within 10 lines based on search results and RAG processing. '
        'Do not include any website URLs.'
    ),
    agent=dermatologist_agent,
    tools=[search_tool, scrape_tool],
    llm=gemini
)


In [ ]:
from crewai import Process
crew = Crew(
    agents=[dermatologist_agent, retriever_agent],
    tasks=[dermatology_task, rag_task],
    verbose=False,
    process=Process.sequential
)


In [ ]:
question = "What are the common treatments for acne?"
inputs = {"question":question}

In [ ]:
results=crew.kickoff(inputs=inputs)
print(results)



> Entering new CrewAgentExecutor chain...
Thought: I need to understand the user's query to provide a relevant and helpful response. 
Action: Read website content
Action Input: {"website_url": "https://www.mayoclinic.org/diseases-conditions/skin-conditions/symptoms-causes/syc-20353632"} 

Binge-eating disorder - Symptoms and causes - Mayo Clinic
This content does not have an English version.This content does not have an Arabic version.
Skip to contentCare at Mayo Clinic Patient-Centered Care About Mayo Clinic Request Appointment Find a Doctor Locations Clinical Trials Connect to Support Groups Patient & Visitor Guide Billing & Insurance Departments & Centers International Services Contact UsPatient & Visitor Guide Health Library Diseases & Conditions Symptoms Tests & Procedures Drugs & Supplements Healthy Lifestyle Books & SubscriptionsDiseases & Conditions For Medical Professionals Medical Professional Resources Refer a Patient Continuing Medical Education AskMayoExpert Mayo Clinic 

KeyboardInterrupt: 